### Relevant Imports

In [1]:
import os
import matplotlib.pyplot as plt
import splitfolders
import tensorflow as tf
import warnings


import cv2
import numpy as np
import dlib
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from scipy.signal import deconvolve

config = tf.compat.v1.ConfigProto()
warnings.filterwarnings("default", category=DeprecationWarning, module=__name__)

2023-09-24 10:50:09.744847: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
